In [2]:
import modulos as md

#lista=md.filtradodemonedas()

lista = ['BCHUSDT', 'XRPUSDT', 'EOSUSDT', 'LTCUSDT', 'TRXUSDT', 'ETCUSDT', 'LINKUSDT', 'XLMUSDT', 'ADAUSDT', 'BNBUSDT', 'ATOMUSDT', 'ALGOUSDT', 
         'KNCUSDT', 'COMPUSDT', 'DOGEUSDT', 'KAVAUSDT', 'WAVESUSDT', 'MKRUSDT', 'SNXUSDT', 'DOTUSDT', 'CRVUSDT', 'SOLUSDT', 'STORJUSDT', 'UNIUSDT', 
         'AVAXUSDT', 'FTMUSDT', 'TOMOUSDT', 'NEARUSDT', 'AAVEUSDT', 'FILUSDT', 'MATICUSDT', 'AXSUSDT', 'SANDUSDT', 'XEMUSDT', 'MANAUSDT', 'LINAUSDT', 
         'MTLUSDT', '1000SHIBUSDT', 'MASKUSDT', 'DYDXUSDT', '1000XECUSDT', 'GALAUSDT', 'FLOWUSDT', 'GMTUSDT', 'APEUSDT', 'OPUSDT', 'INJUSDT', 'LDOUSDT', 
         'APTUSDT', 'FETUSDT', 'RNDRUSDT', 'AGIXUSDT', 'PHBUSDT', 'CFXUSDT', 'STXUSDT', 'LQTYUSDT', 'ARBUSDT', 'SUIUSDT', '1000PEPEUSDT', 'XVGUSDT']

#lista=['SKLUSDT']

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
###################################### FILTRADO DE MONEDAS PARA ESTRATEGIA SANTAAAAAA
###################################################################################################################
for symbol in lista:
    try:
        data = md.estrategia_santa(symbol, tp_flag = True)
        resultado = md.backtesting(data, plot_flag = False)
        #if (resultado['Profit Factor']>1 or resultado['Return [%]'] >0) and resultado['Win Rate [%]']>50:
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)} - Win Rate [%]: {md.truncate(resultado['Win Rate [%]'],2)}")
    except Exception as falla:
        pass


In [ ]:
###################################### FILTRADO DE MONEDAS PARA ESTRATEGIA BB
###################################################################################################################
for symbol in lista:
    try:
        data = md.estrategia_bb(symbol, tp_flag = True)
        resultado = md.backtesting(data, plot_flag = False)
        #if resultado['Profit Factor'] > 1 and resultado[6]/resultado[17] >=0.3:
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)}")
    except Exception as falla:
        pass

In [3]:
###################################### FILTRADO DE MONEDAS PARA ESTRATEGIA BB
###################################################################################################################
for symbol in lista:
    try:
        data = md.estrategia_variacion_precio(symbol)
        resultado = md.backtesting(data, plot_flag = False)
        #if resultado['Profit Factor'] > 1 and resultado[6]/resultado[17] >=0.3:
        print(f"{symbol} - Return [%]: {md.truncate(resultado['Return [%]'],2)}% - # Trades: {resultado['# Trades']} - Profit Factor: {md.truncate(resultado['Profit Factor'],2)}")
    except Exception as falla:
        pass

c:\LUCHO\personal\repopersonal\luchocrypto\FENIX\modulos.py:462: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, Fenix, cash=1000)


In [1]:
import modulos as md
data = md.estrategia_variacion_precio('LTCUSDT')
resultado = md.backtesting(data, plot_flag = False)

C:\Users\Luis Gomez Freites\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data

,Open Time,Open,High,Low,Close,Volume,ema20,ema50,ema200,atr,upper,lower,avg_volume,n_atr,signal,take_profit,stop_loss
timestamp,,,,,,,,,,,,,,,,,
2023-06-03 04:00:00,2023-06-03 04:00:00,94.56,94.81,94.49,94.74,9013.019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,0,96.6348,92.8452
2023-06-03 05:00:00,2023-06-03 05:00:00,94.73,94.83,94.41,94.45,10050.932,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,0,NaN,NaN
2023-06-03 06:00:00,2023-06-03 06:00:00,94.45,95.11,94.45,95.01,11533.983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,0,NaN,NaN
2023-06-03 07:00:00,2023-06-03 07:00:00,95.02,95.14,94.73,94.84,12389.069,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,0,NaN,NaN
2023-06-03 08:00:00,2023-06-03 08:00:00,94.85,95.26,94.77,94.95,14198.924,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-07-14 15:00:00,2023-07-14 15:00:00,101.01,101.44,100.67,100.95,33881.656,100.977055,99.997994,98.606776,1.282101,102.967431,99.645569,43492.16845,50,0,NaN,NaN
2023-07-14 16:00:00,2023-07-14 16:00:00,100.94,102.05,99.26,100.35,96837.052,100.917335,100.011798,98.624122,1.389808,102.961157,99.540843,46177.09600,50,0,NaN,NaN
2023-07-14 17:00:00,2023-07-14 17:00:00,100.34,100.34,96.50,96.94,143733.272,100.538541,99.891336,98.607364,1.565536,103.594603,98.602397,49517.00135,50,0,NaN,NaN
